In [1]:
# this is a little trick to make sure the the notebook takes up most of the screen:
from IPython.display import HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Recommendation to leave the logging config like this, otherwise you'll be flooded with unnecessary info
import logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')  


# Recommendation: logging config like this, otherwise you'll be flooded with unnecessary information
import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
# import all modeling concepts
from crestdsl.model import *

# import the simulator
from crestdsl.simulation import Simulator

# import the plotting libraries that can visualise the CREST systems
from crestdsl.ui import elk

# we will create tests for each Entity
import unittest

class TestClass(unittest.TestCase):
    @classmethod
    def runall(cls):
        tests = unittest.TestLoader().loadTestsFromTestCase(cls)
        return unittest.TextTestRunner().run(tests)

In [3]:
class Resources(object):
    electricity = Resource("Watt", REAL)
    switch = Resource("switch", ["on", "off"])
    pourcent = Resource("factor", REAL) 
    light = Resource("Lumen", INTEGER)
    time = Resource("minutes", REAL)
    water = Resource("liter", REAL)
    celsius = Resource("Celsius", REAL)
    onOffAuto = Resource("onOffAutoSwitch", ["on", "off", "auto"])
    integer = Resource("integer", INTEGER)
    weight = Resource("kg", REAL)
    lenght = Resource("m", REAL)
    area = Resource("m²", REAL)
    luminous_efficacy = Resource("lm/W", REAL)

In [4]:
# A battery that recharges if it's not full and that supply electricity if needed. Values are changeable

class Battery(Entity):
    
    """- - - - - - - - - Constructor - - - - - - - - -"""
    
    def __init__(self, max_charge=10000, usage_limit=30):

        self.maximal_charge.value = max_charge
        self.threshold.value = self.maximal_charge.value * usage_limit/100    
    
    """ - - - - - - - PORTS - - - - - - - - - - """
    panel_electricity = Input(resource=Resources.electricity, value=400)
    battery_out = Output(resource=Resources.electricity, value=0)
    switch_power = Output(resource=Resources.switch, value="on") #Initial state is Recharge, Power Grid will start the supply
    
    maximal_charge = Local(resource=Resources.electricity, value=10000) # Max charge of the Battery
    threshold = Local(resource=Resources.electricity, value=0.3*maximal_charge.value) # 30% * 10 000 Watt
    charge = Local(resource=Resources.electricity, value=1000) # 10% at the begginning 
    consumption = Local(resource=Resources.electricity, value=300) # Local only for this test
    
    """ - - - - - - - SUBENTITIES - - - - - - - - - - """
    #None
    
    """ - - - - - - - INFLUENCES - - - - - - - - - - """
    #None
    
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    supplyAndRecharge = State()
    recharge = current = State()
    supply = State()
    
    supply_to_SupplyAndRecharge = Transition(source=supply, target=supplyAndRecharge, guard=(lambda self: self.panel_electricity.value > self.consumption.value))
    supply_to_Recharge = Transition(source=supply, target=recharge, guard=(lambda self: self.charge.value < self.maximal_charge.value * 0.1))
    
    supplyAndRecharge_to_Supply = Transition(source=supplyAndRecharge, target=supply,guard=(lambda self: self.panel_electricity.value < self.consumption.value))
    
    recharge_to_SupplyAndRecharge = Transition(source=recharge, target=supplyAndRecharge, guard=(lambda self: self.charge.value > self.threshold.value and self.panel_electricity.value > self.consumption.value))
    
    recharge_to_Supply = Transition(source=recharge, target=supply, guard=(lambda self: self.charge.value > self.threshold.value and self.panel_electricity.value <= self.consumption.value))
    
    """ - - - - - - - UPDATES - - - - - - - - - - """

    #Electricity Uptdates (Charge and Battery_out)
    
    @update(state=supplyAndRecharge, target=battery_out)
    def update_battery_out_SupplyAndRecharge(self, dt):
        return self.consumption.value
    
    @update(state=supplyAndRecharge, target=charge)
    def update_charge_SupplyAndRecharge(self, dt):
        return min(self.charge.value+(self.panel_electricity.value-self.consumption.value)*dt,self.maximal_charge.value)
    
    @update(state=supply, target=battery_out)
    def update_battery_out_Supply(self, dt):
        return self.consumption.value
    
    @update(state=supply, target=charge)
    def update_charge_Supply(self, dt):
        return self.charge.value+(self.panel_electricity.value-self.consumption.value)*dt
    
    @update(state=recharge, target=battery_out)
    def update_battery_out_Recharge(self, dt):
        return 0
    
    @update(state=recharge, target=charge)
    def update_charge_Recharge(self, dt):
        return self.charge.value+self.panel_electricity.value*dt
    
    #Power_Grid Switch Updates
    
    @update(state=supplyAndRecharge, target=switch_power)
    def update_switch_power_SupplyAndRecharge(self, dt):
        return "off"
    
    @update(state=supply, target=switch_power)
    def update_switch_power_Supply(self, dt):
        return "off"
    
    @update(state=recharge, target=switch_power)
    def update_switch_power_Recharge(self, dt):
        return "on"
    
elk.plot(Battery())

In [5]:
#Power grid can supply any amount of electricity needed

class Power_Grid(Entity):

    """ - - - - - - - PORTS - - - - - - - - - - """    
    
    switch_PG = Input(resource=Resources.switch, value="off") 
    
    electricity_Supply = Output(resource=Resources.electricity, value=0)
    
    consumption = Local(Resources.electricity, 300)

    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """

    state = current = State() # the only state of this entity
    
    """ - - - - - - - UPDATES - - - - - - - - - - """

    @update(state=state, target=electricity_Supply)
    def power_grid_output(self, dt):
        # When the lamp is on, then we convert electricity to temperature at a rate of 100Watt = 1Celsius
        if self.switch_PG.value == "on":
            return self.consumption.value
        else:
            return 0

# show us what it looks like
elk.plot(Power_Grid())

In [6]:
# Supply with the battery if it's not empty, otherwise change to the grid

class ElectricPart(Entity):
    
    """ - - - - - - - PORTS - - - - - - - - - - """
    panel_elec = Input(resource=Resources.electricity, value=400)
    
    elec_out = Output(resource=Resources.electricity, value=0) 
    
    """ - - - - - - - SUBENTITIES - - - - - - - - - - """
    battest = Battery()
    testGrid = Power_Grid()
    
    
    """ - - - - - - - INFLUENCES - - - - - - - - - - """
    
    panel_elec_propagation = Influence(source=panel_elec, target = battest.panel_electricity)
    switch_power_propagation = Influence(battest.switch_power, testGrid.switch_PG)
    
    
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    state = current = State()
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
    
    @update(state=state, target=elec_out)
    def update_elec_out(self, dt):
        # no electricity
        return self.battest.battery_out.value + self.testGrid.electricity_Supply.value

# create an instance!
elk.plot(ElectricPart())

In [7]:
#Test Battery, at different moments to test if the supply is everytime made by the right source

class ElectricPartTest(TestClass):
    def test_battery_500(self):
        need_watt = 400
        elecSys1 = ElectricPart()
        elecSys1.battest.consumption.value = need_watt
        elecSys1.testGrid.consumption.value = need_watt
        elecSys1.panel_elec.value = 500
        
        sim1 = Simulator(elecSys1)
        sim1.stabilise()

        self.assertEqual(elecSys1.battest.battery_out.value, 0)
        self.assertEqual(elecSys1.testGrid.electricity_Supply.value, need_watt)
        self.assertEqual(elecSys1.battest.current, elecSys1.battest.recharge)


    def test_battery_504(self):
        need_watt = 400
        elecSys2 = ElectricPart()
        elecSys2.battest.consumption.value = need_watt
        elecSys2.testGrid.consumption.value = need_watt
        elecSys2.panel_elec.value = 500
        
        sim2 = Simulator(elecSys2)
        sim2.stabilise()
        sim2.advance(4)
        
        self.assertEqual(elecSys2.battest.battery_out.value, 0)
        self.assertEqual(elecSys2.testGrid.electricity_Supply.value, need_watt)
        self.assertEqual(elecSys2.battest.charge.value, 3000)
        self.assertEqual(elecSys2.battest.current, elecSys2.battest.recharge)
        
    def test_battery_505(self):
        need_watt = 400
        elecSys3 = ElectricPart()
        elecSys3.battest.consumption.value = need_watt
        elecSys3.testGrid.consumption.value = need_watt
        elecSys3.panel_elec.value = 500
        
        sim3 = Simulator(elecSys3)
        sim3.stabilise()
        sim3.advance(5)
        
        self.assertEqual(elecSys3.battest.battery_out.value, need_watt)
        self.assertEqual(elecSys3.testGrid.electricity_Supply.value, 0)
        self.assertAlmostEqual(elecSys3.battest.charge.value, 3100)
        self.assertEqual(elecSys3.battest.current, elecSys3.battest.supplyAndRecharge)    
        
    def test_battery_578(self):
        need_watt = 400
        elecSys4 = ElectricPart()
        elecSys4.battest.consumption.value = need_watt
        elecSys4.testGrid.consumption.value = need_watt
        elecSys4.panel_elec.value = 500
        
        sim4 = Simulator(elecSys4)
        sim4.stabilise()
        sim4.advance(78)
        
        self.assertEqual(elecSys4.battest.battery_out.value, need_watt)
        self.assertEqual(elecSys4.testGrid.electricity_Supply.value, 0)
        self.assertEqual(elecSys4.battest.current, elecSys4.battest.supplyAndRecharge)
               
    def test_battery_300(self):
        need_watt = 400
        elecSys10 = ElectricPart()
        elecSys10.battest.consumption.value = need_watt
        elecSys10.testGrid.consumption.value = need_watt
        elecSys10.panel_elec.value = 300
        elecSys10.battest.charge.value = 4000
        
        sim10 = Simulator(elecSys10)
        sim10.stabilise()
        
        self.assertEqual(elecSys10.battest.battery_out.value, need_watt)
        self.assertEqual(elecSys10.testGrid.electricity_Supply.value, 0)
        self.assertEqual(elecSys10.battest.current, elecSys10.battest.supply)
        
    def test_battery_330(self):
        need_watt = 400
        elecSys11 = ElectricPart()
        elecSys11.battest.consumption.value = need_watt
        elecSys11.testGrid.consumption.value = need_watt
        elecSys11.panel_elec.value = 300
        elecSys11.battest.charge.value = 4000
        
        sim11 = Simulator(elecSys11)
        sim11.stabilise()
        sim11.advance(30)

        self.assertEqual(elecSys11.battest.battery_out.value, need_watt)
        self.assertEqual(elecSys11.testGrid.electricity_Supply.value, 0)
        self.assertEqual(elecSys11.battest.charge.value, 1000)
        self.assertEqual(elecSys11.battest.current, elecSys11.battest.supply)
        
        
    def test_battery_331(self):
        need_watt = 400
        elecSys12 = ElectricPart()
        elecSys12.battest.consumption.value = need_watt
        elecSys12.testGrid.consumption.value = need_watt
        elecSys12.panel_elec.value = 300
        elecSys12.battest.charge.value = 4000
        
        sim12 = Simulator(elecSys12)
        sim12.stabilise()
        sim12.advance(31)
        
        self.assertEqual(elecSys12.battest.battery_out.value, 0)
        self.assertEqual(elecSys12.testGrid.electricity_Supply.value, need_watt)
        self.assertAlmostEqual(elecSys12.battest.charge.value, 1300)
        self.assertEqual(elecSys12.battest.current, elecSys12.battest.recharge)
        
ElectricPartTest.runall()




.......
----------------------------------------------------------------------
Ran 7 tests in 2.531s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>

In [11]:
#Component made for a presentation, can be deleted

class Component(Entity):
    """ - - - - - - - PORTS - - - - - - - - - - """    
    Input = Input(resource=Resources.switch, value="off") 
    
    Local = Local(Resources.electricity, 300)

    Output = Output(resource=Resources.water, value=15.5) 
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    state1 = current = State() 
    state2 = State()
    
    state1_to_state2 = Transition(source=state1, target=state2,
                    guard=(lambda self: self.Input.value == "on"))
    
    state2_to_state1 = Transition(source=state2, target=state1,
                    guard=(lambda self: self.Input.value == "off"))
    
    

    """ - - - - - - - UPDATES - - - - - - - - - - """

    @update(state=state2, target=Output)
    def update_output_state2(self, dt):
        return self.Local.value
    
    
#show us what it looks like
elk.plot(Component())

#com = Component()
#com.switch_Input.value="on"
#sim = Simulator(com)
#sim.stabilize()
#sim.plot()

In [9]:
# Solar panel that change the luminous energy that is send to make electricity

class SolarPanel(Entity):
    """ - - - - - - - PORTS - - - - - - - - - - """    
    lux_In = Input(resource=Resources.light, value=0)
    
    size_Panel = Local(Resources.area, 20)
    luminous_Efficacy = Local(Resources.luminous_efficacy,110)
    lumen_In = Local(Resources.light, 0)

    electricity_Out = Output(resource=Resources.electricity, value=0) 
    
    def __init__(self, size_Panels=0):
        self.size_Panel.value = size_Panels

    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    
    state = current = State() 

    """ - - - - - - - UPDATES - - - - - - - - - - """
    
    @update(state=state, target=lumen_In)
    def update_lumen_In(self, dt):
        return self.lux_In.value*self.size_Panel.value
    
    @update(state=state, target=electricity_Out)
    def update_electricity_Out(self, dt):
        return self.lux_In.value*self.size_Panel.value/114
    
#show us what it looks like
elk.plot(SolarPanel())

#panel = SolarPanel()
#sim = Simulator(panel)
#sim.stabilize()
#sim.plot()